In [28]:
import torch
import os
from torchmetrics.wrappers import MultioutputWrapper
import torchmetrics
from copy import deepcopy
import numpy as np
from qtaim_embed.models.utils import load_graph_level_model_from_config
from qtaim_embed.utils.tests import get_dataset_graph_level
from qtaim_embed.data.dataloader import DataLoaderMoleculeGraphTask
from qtaim_embed.utils.data import get_default_graph_level_config
from qtaim_embed.core.dataset import HeteroGraphGraphLabelDataset
from qtaim_embed.models.utils import get_test_train_preds_as_df, test_and_predict_libe


In [13]:
model_root = "../../../../data/saved_models/final_set/qm8/"
model_log_root = "../../../../data/saved_models/final_set/qm8/no_log/"
model_list = os.listdir(model_root)
model_log_list = os.listdir(model_log_root)


In [18]:
for ind, model in enumerate(model_list):
    # load model
    model_loc = model_root + model
    try:
        model_config = {"model": {"restore": True, "restore_path": model_loc}}
        model = load_graph_level_model_from_config(model_config["model"])
        print(model.hparams["atom_input_size"], model.hparams["bond_input_size"])
    except:
        pass

print("---" * 30)

for ind, model in enumerate(model_log_list):
    # load model
    model_loc = model_log_root + model
    try:
        model_config = {"model": {"restore": True, "restore_path": model_loc}}
        model = load_graph_level_model_from_config(model_config["model"])
        print(model.hparams["atom_input_size"], model.hparams["bond_input_size"])
    except:
        pass


:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 42
... > number of tasks: 2
:::MODEL LOADED:::
34 29
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 42
... > number of tasks: 2
:::MODEL LOADED:::
34 29
:::RESTORING MODEL FROM EXISTING FILE:::
:::NO MODEL FOUND LOADING FRESH MODEL:::
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 90
... > number of tasks: 2
:::MODEL LOADED:::
34 29
------------------------------------------------------------------------------------------
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 82
... > number of tasks: 2
:::MODEL LOADED:::
30 24
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 150
... > number of tasks: 2
:::MODEL LOADED:::
30 24
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 42
... > number of tasks: 2
:::MODEL LOADED:::
30 24
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 90
... > number of tasks: 2
:::MODEL LOADED:::
30 24
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size

In [19]:
feat_dict_large = {
    "atom": [
        "extra_feat_atom_Lagrangian_K",
        "extra_feat_atom_Hamiltonian_K",
        "extra_feat_atom_e_density",
        "extra_feat_atom_lap_e_density",
        "extra_feat_atom_e_loc_func",
        "extra_feat_atom_ave_loc_ion_E",
        "extra_feat_atom_delta_g_promolecular",
        "extra_feat_atom_delta_g_hirsh",
        "extra_feat_atom_esp_nuc",
        "extra_feat_atom_esp_e",
        "extra_feat_atom_esp_total",
        "extra_feat_atom_grad_norm",
        "extra_feat_atom_lap_norm",
        "extra_feat_atom_eig_hess",
        "extra_feat_atom_det_hessian",
        "extra_feat_atom_ellip_e_dens",
        "extra_feat_atom_eta",
        "extra_feat_atom_energy_density",
        "extra_feat_atom_density_beta",
        "extra_feat_atom_density_alpha",
        "extra_feat_atom_spin_density",
        "extra_feat_atom_lol",
    ],
    "bond": [
        "bond_length",
        "extra_feat_bond_Lagrangian_K",
        "extra_feat_bond_Hamiltonian_K",
        "extra_feat_bond_e_density",
        "extra_feat_bond_lap_e_density",
        "extra_feat_bond_e_loc_func",
        "extra_feat_bond_ave_loc_ion_E",
        "extra_feat_bond_delta_g_promolecular",
        "extra_feat_bond_delta_g_hirsh",
        "extra_feat_bond_esp_nuc",
        "extra_feat_bond_esp_e",
        "extra_feat_bond_esp_total",
        "extra_feat_bond_grad_norm",
        "extra_feat_bond_lap_norm",
        "extra_feat_bond_eig_hess",
        "extra_feat_bond_det_hessian",
        "extra_feat_bond_ellip_e_dens",
        "extra_feat_bond_eta",
        "extra_feat_bond_energy_density",
        "extra_feat_bond_density_beta",
        "extra_feat_bond_density_alpha",
        "extra_feat_bond_spin_density",
        "extra_feat_bond_lol",
    ],
    "global": ["E1-CC2", "E2-CC2"],
}

feat_dict = {
    "atom": [
        "extra_feat_atom_Hamiltonian_K",
        "extra_feat_atom_e_density",
        "extra_feat_atom_lap_e_density",
        "extra_feat_atom_e_loc_func",
        "extra_feat_atom_ave_loc_ion_E",
        "extra_feat_atom_delta_g_promolecular",
        "extra_feat_atom_delta_g_hirsh",
        "extra_feat_atom_esp_nuc",
        "extra_feat_atom_esp_e",
        "extra_feat_atom_esp_total",
        "extra_feat_atom_grad_norm",
        "extra_feat_atom_lap_norm",
        "extra_feat_atom_eig_hess",
        "extra_feat_atom_det_hessian",
        "extra_feat_atom_ellip_e_dens",
        "extra_feat_atom_eta",
        "extra_feat_atom_energy_density",
        "extra_feat_atom_lol",
    ],
    "bond": [
        "bond_length",
        "extra_feat_bond_Lagrangian_K",
        "extra_feat_bond_Hamiltonian_K",
        "extra_feat_bond_e_density",
        "extra_feat_bond_lap_e_density",
        "extra_feat_bond_e_loc_func",
        "extra_feat_bond_ave_loc_ion_E",
        "extra_feat_bond_delta_g_promolecular",
        "extra_feat_bond_delta_g_hirsh",
        "extra_feat_bond_esp_nuc",
        "extra_feat_bond_esp_e",
        "extra_feat_bond_esp_total",
        "extra_feat_bond_grad_norm",
        "extra_feat_bond_lap_norm",
        "extra_feat_bond_eig_hess",
        "extra_feat_bond_det_hessian",
        "extra_feat_bond_ellip_e_dens",
        "extra_feat_bond_eta",
    ],
    "global": ["E1-CC2", "E2-CC2"],
}


In [20]:
qm8_loc = "../../../../data/splits_1205/test_qm8_qtaim_1205_labelled.pkl"
qm8_train_loc = "../../../../data/splits_1205/train_qm8_qtaim_1205_labelled.pkl"

dataset_test_qtaim_small = HeteroGraphGraphLabelDataset(
    file=qm8_loc,
    allowed_ring_size=[4, 5, 6, 7],
    allowed_charges=None,
    allowed_spins=None,
    self_loop=True,
    extra_keys=feat_dict,
    target_list=["E1-CC2", "E2-CC2"],
    extra_dataset_info={},
    debug=False,
    log_scale_features=False,
    log_scale_targets=False,
    standard_scale_features=True,
    standard_scale_targets=True,
)
dataset_train_qtaim_small = HeteroGraphGraphLabelDataset(
    file=qm8_train_loc,
    allowed_ring_size=[4, 5, 6, 7],
    allowed_charges=None,
    allowed_spins=None,
    self_loop=True,
    extra_keys=feat_dict,
    target_list=["E1-CC2", "E2-CC2"],
    extra_dataset_info={},
    debug=False,
    log_scale_features=False,
    log_scale_targets=False,
    standard_scale_features=True,
    standard_scale_targets=True,
)

dataset_test_qtaim_large = HeteroGraphGraphLabelDataset(
    file=qm8_loc,
    allowed_ring_size=[4, 5, 6, 7],
    allowed_charges=None,
    allowed_spins=None,
    self_loop=True,
    extra_keys=feat_dict_large,
    target_list=["E1-CC2", "E2-CC2"],
    extra_dataset_info={},
    debug=False,
    log_scale_features=True,
    log_scale_targets=False,
    standard_scale_features=True,
    standard_scale_targets=True,
)
dataset_train_qtaim_large = HeteroGraphGraphLabelDataset(
    file=qm8_train_loc,
    allowed_ring_size=[4, 5, 6, 7],
    allowed_charges=None,
    allowed_spins=None,
    self_loop=True,
    extra_keys=feat_dict_large,
    target_list=["E1-CC2", "E2-CC2"],
    extra_dataset_info={},
    debug=False,
    log_scale_features=True,
    log_scale_targets=False,
    standard_scale_features=True,
    standard_scale_targets=True,
)


... > creating MoleculeWrapper objects


100%|██████████| 2178/2178 [00:00<00:00, 3171.32it/s]


... > bond_feats_error_count:  0
... > atom_feats_error_count:  0
element set {'H', 'O', 'N', 'C', 'F'}
selected atomic keys ['extra_feat_atom_Hamiltonian_K', 'extra_feat_atom_e_density', 'extra_feat_atom_lap_e_density', 'extra_feat_atom_e_loc_func', 'extra_feat_atom_ave_loc_ion_E', 'extra_feat_atom_delta_g_promolecular', 'extra_feat_atom_delta_g_hirsh', 'extra_feat_atom_esp_nuc', 'extra_feat_atom_esp_e', 'extra_feat_atom_esp_total', 'extra_feat_atom_grad_norm', 'extra_feat_atom_lap_norm', 'extra_feat_atom_eig_hess', 'extra_feat_atom_det_hessian', 'extra_feat_atom_ellip_e_dens', 'extra_feat_atom_eta', 'extra_feat_atom_energy_density', 'extra_feat_atom_lol']
selected bond keys ['bond_length', 'extra_feat_bond_Lagrangian_K', 'extra_feat_bond_Hamiltonian_K', 'extra_feat_bond_e_density', 'extra_feat_bond_lap_e_density', 'extra_feat_bond_e_loc_func', 'extra_feat_bond_ave_loc_ion_E', 'extra_feat_bond_delta_g_promolecular', 'extra_feat_bond_delta_g_hirsh', 'extra_feat_bond_esp_nuc', 'extra_fe

100%|██████████| 2178/2178 [00:04<00:00, 531.20it/s]


included in labels
{'global': ['E1-CC2', 'E2-CC2']}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_H', 'chemical_symbol_O', 'chemical_symbol_N', 'chemical_symbol_C', 'chemical_symbol_F', 'extra_feat_atom_Hamiltonian_K', 'extra_feat_atom_e_density', 'extra_feat_atom_lap_e_density', 'extra_feat_atom_e_loc_func', 'extra_feat_atom_ave_loc_ion_E', 'extra_feat_atom_delta_g_promolecular', 'extra_feat_atom_delta_g_hirsh', 'extra_feat_atom_esp_nuc', 'extra_feat_atom_esp_e', 'extra_feat_atom_esp_total', 'extra_feat_atom_grad_norm', 'extra_feat_atom_lap_norm', 'extra_feat_atom_eig_hess', 'extra_feat_atom_det_hessian', 'extra_feat_atom_ellip_e_dens', 'extra_feat_atom_eta', 'extra_feat_atom_energy_density', 'extra_feat_atom_lol'], 'bond': ['metal bond', 'ring inclusion', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length', 'extra_feat_bond_Lagrangian_K', 'extra_feat_bond

...

... > bond_feats_error_count:  0
... > atom_feats_error_count:  0
element set {'H', 'O', 'N', 'C', 'F'}
selected atomic keys ['extra_feat_atom_Lagrangian_K', 'extra_feat_atom_Hamiltonian_K', 'extra_feat_atom_e_density', 'extra_feat_atom_lap_e_density', 'extra_feat_atom_e_loc_func', 'extra_feat_atom_ave_loc_ion_E', 'extra_feat_atom_delta_g_promolecular', 'extra_feat_atom_delta_g_hirsh', 'extra_feat_atom_esp_nuc', 'extra_feat_atom_esp_e', 'extra_feat_atom_esp_total', 'extra_feat_atom_grad_norm', 'extra_feat_atom_lap_norm', 'extra_feat_atom_eig_hess', 'extra_feat_atom_det_hessian', 'extra_feat_atom_ellip_e_dens', 'extra_feat_atom_eta', 'extra_feat_atom_energy_density', 'extra_feat_atom_density_beta', 'extra_feat_atom_density_alpha', 'extra_feat_atom_spin_density', 'extra_feat_atom_lol']
selected bond keys ['bond_length', 'extra_feat_bond_Lagrangian_K', 'extra_feat_bond_Hamiltonian_K', 'extra_feat_bond_e_density', 'extra_feat_bond_lap_e_density', 'extra_feat_bond_e_loc_func', 'extra_feat_b

100%|██████████| 19607/19607 [00:36<00:00, 530.74it/s]


included in labels
{'global': ['E1-CC2', 'E2-CC2']}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_H', 'chemical_symbol_O', 'chemical_symbol_N', 'chemical_symbol_C', 'chemical_symbol_F', 'extra_feat_atom_Lagrangian_K', 'extra_feat_atom_Hamiltonian_K', 'extra_feat_atom_e_density', 'extra_feat_atom_lap_e_density', 'extra_feat_atom_e_loc_func', 'extra_feat_atom_ave_loc_ion_E', 'extra_feat_atom_delta_g_promolecular', 'extra_feat_atom_delta_g_hirsh', 'extra_feat_atom_esp_nuc', 'extra_feat_atom_esp_e', 'extra_feat_atom_esp_total', 'extra_feat_atom_grad_norm', 'extra_feat_atom_lap_norm', 'extra_feat_atom_eig_hess', 'extra_feat_atom_det_hessian', 'extra_feat_atom_ellip_e_dens', 'extra_feat_atom_eta', 'extra_feat_atom_energy_density', 'extra_feat_atom_density_beta', 'extra_feat_atom_density_alpha', 'extra_feat_atom_spin_density', 'extra_feat_atom_lol'], 'bond': ['metal bond', 'ring inclus

100%|██████████| 19607/19607 [00:00<00:00, 34016.36it/s]


original loader node types: dict_keys(['atom', 'bond', 'global'])
original loader label types: dict_keys(['global'])
... > Log scaling features
... > Log scaling features complete
... > Scaling features
mean [ 6.94480210e-01  1.98050807e-01  3.01505867e-02  1.64613505e-02
  1.04097064e-02  2.77211426e-03  5.07415527e-04  3.58360568e-01
  5.44933523e-02  4.10984611e-02  2.38230492e-01  9.64309161e-04
  1.32616490e+00  7.19627663e+00 -7.19627663e+00 -8.52669674e+00
  6.93095289e-01  1.42630408e+00  7.98208921e-02  1.27152467e-01
  9.14825234e+00 -3.00580327e+00  8.98075974e+00  0.00000000e+00
 -8.52669674e+00 -8.52669674e+00 -2.22241076e+01  3.32077713e-04
 -7.17475531e-01 -7.19627663e+00  2.19560489e+00  2.19560489e+00
  0.00000000e+00  6.89766178e-01]
std [5.93734382e-01 3.66119205e-01 1.41385064e-01 1.05542232e-01
 8.43035983e-02 4.37469836e-02 1.87471644e-02 3.46373096e-01
 1.86553982e-01 1.63701555e-01 3.29203625e-01 2.58356011e-02
 1.47188583e+00 5.38900363e+00 5.38900363e+00 5.442

In [22]:
dataset_train_qtaim_large.feature_size()


{'atom': 34, 'bond': 29, 'global': 3}

In [24]:
dataset_test_qtaim_small.feature_size()


{'atom': 30, 'bond': 24, 'global': 3}

In [61]:
results_dict = {
    "qtaim_mid": {"test_r2": 0.0},
    "qtaim_full": {"test_r2": 0.0},
}

str_dataset = "qtaim_large"

for ind, model in enumerate(model_list):
    print(ind)

    try:
        model_config = {"model": {"restore": True, "restore_path": model_loc}}
        model = load_graph_level_model_from_config(model_config["model"])
        model.cpu()

        atom_size = model.hparams["atom_input_size"]
        bond_size = model.hparams["bond_input_size"]
        dataset_test_base = dataset_test_qtaim_small
        dataset_train_base = dataset_train_qtaim_small
        (
            preds_test,
            preds_train,
            label_list,
            label_list_train,
            val_results,
        ) = test_and_predict(dataset_test_base, dataset_train_base, model)
        print(val_results["statistics_dict"]["test"]["r2"])
        if (
            val_results["statistics_dict"]["test"]["r2"]
            > results_dict[str_dataset]["test_r2"]
        ):
            results_dict[str_dataset]["test_r2"] = val_results["statistics_dict"][
                "test"
            ]["r2"]
            results_dict[str_dataset]["test_preds"] = preds_test
            results_dict[str_dataset]["test_labels"] = label_list
            results_dict[str_dataset]["train_preds"] = preds_train
            results_dict[str_dataset]["train_labels"] = label_list_train
            results_dict[str_dataset]["model_loc"] = model_loc
            results_dict[str_dataset]["model"] = model
            results_dict[str_dataset]["statistics_dict"] = val_results
    except:
        pass


0
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 82
... > number of tasks: 2
:::MODEL LOADED:::
----------------------------------------------------------------------------------------------------
Performance training set:	 r2: [0.6236558  0.62387145]	 mae: [0.01723878 0.01460461]	 
Performance test set:	 r2: [0.34214914 0.42779076]	 mae: [0.02521775 0.01844789]
----------------------------------------------------------------------------------------------------
1
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 82
... > number of tasks: 2
:::MODEL LOADED:::
----------------------------------------------------------------------------------------------------
Performance training set:	 r2: [0.620824   0.61974573]	 mae: [0.01733071 0.01465529]	 
Performance test set:	 r2: [0.32171518 0.4104892 ]	 mae: [0.0256028  0.01866389]
----------------------------------------------------------------------------------------------------
2
:::RESTORING MODEL FROM EXISTING FILE:::
reado

In [49]:
results_dict


{'qtaim_mid': {'test_r2': 0.0}, 'qtaim_full': {'test_r2': 0.0}}

In [36]:
import matplotlib.pyplot as plt
import seaborn as sns


In [45]:
df_test, df_train = get_test_train_preds_as_df(results_dict, "qtaim_mid")

# specify number of bins
num_bins = 50
# make jointplots
sns.jointplot(
    x="preds",
    y="labels",
    data=df_test,
    kind="reg",
    height=20,
    space=0,
    ylim=(-10, 15),
    xlim=(-10, 15),
    # joint_kws={"line_kws": {"color": "red"}},
    # joint_kws={"gridsize": 80},
).set_axis_labels("Target(eV)", "Predicted(eV)")
# set font sizes
plt.xlabel("Target(eV)", fontsize=40)
plt.ylabel("Predicted(eV)", fontsize=40)
plt.xticks(fontsize=26)
plt.yticks(fontsize=26)
# rotate xticks
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
# add diagonal line
plt.plot([-10, 15], [-10, 15], linewidth=5, color="red")
# add statistics
plt.text(
    -9,
    14,
    "$R^2$: {:.2f}".format(results_dict["qtaim_mid"]["statistics_dict"]["test"]["r2"]),
    fontsize=30,
)


# add title
plt.show()


KeyError: 'test_preds'

In [58]:
print("---" * 30)

for ind, model in enumerate(model_log_list):
    # load model
    model_loc = model_log_root + model
    model_config = {"model": {"restore": True, "restore_path": model_loc}}
    model = load_graph_level_model_from_config(model_config["model"])
    model.cpu()
    print(model.hparams["atom_input_size"], model.hparams["bond_input_size"])

    atom_size = model.hparams["atom_input_size"]
    bond_size = model.hparams["bond_input_size"]
    dataset_test_base = dataset_test_qtaim_small
    dataset_train_base = dataset_train_qtaim_small
    (
        preds_test,
        preds_train,
        label_list,
        label_list_train,
        val_results,
    ) = test_and_predict(dataset_test_base, dataset_train_base, model)


------------------------------------------------------------------------------------------
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 82
... > number of tasks: 2
:::MODEL LOADED:::
30 24
----------------------------------------------------------------------------------------------------
Performance training set:	 r2: [0.7061301 0.63502  ]	 mae: [0.01606936 0.01499428]	 
Performance test set:	 r2: [-0.17608178  0.15907931]	 mae: [0.0368638  0.02402486]
----------------------------------------------------------------------------------------------------
:::RESTORING MODEL FROM EXISTING FILE:::
readout in size 150
... > number of tasks: 2
:::MODEL LOADED:::
30 24
----------------------------------------------------------------------------------------------------
Performance training set:	 r2: [0.7656634 0.7428179]	 mae: [0.01463437 0.01218217]	 
Performance test set:	 r2: [0.05190825 0.29930967]	 mae: [0.03247513 0.02111006]
---------------------------------------------------